In [2]:
import math
import tkinter as tk
from tkinter import ttk
import threading
import time
import random
import numpy as np
import socket
import os

from scapy.all import Ether, wrpcap

import velodyne_decoder as vd

from PIL import Image, ImageTk
from PIL.Image import Resampling

import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from mpl_toolkits.mplot3d import Axes3D

import pygame

###############################################################################
#                                GLOBALS                                      #
###############################################################################

exit_flag = threading.Event()  # signals the background game to stop

# LiDAR / Networking
UDP_IP = "192.168.1.2"
UDP_PORTS = [2368]
HEADERS = {
    2368: bytes.fromhex(
        "ff ff ff ff ff ff 60 76 88 34 2d 4f 08 00 45 00 04 d2 00 00 40 00 ff 11 b4 a9 c0 a8 01 c9 ff ff ff ff 09 40 09 3e 04 be 00 00"
    )
}

BASE_FOLDER_PATH = "C:/PickHacks/PickHacks/"
PLAYER1_FOLDER = os.path.join(BASE_FOLDER_PATH, "Player1")
PLAYER2_FOLDER = os.path.join(BASE_FOLDER_PATH, "Player2")

# Audio files
INTRO_MUSIC = "1.mp3"         # played for the initial 10s
IDLE_MUSIC = "2.mp3"          # idle music
GAME_SOUND = "Game Sound.wav" # "Run!!" state

# Image assets
start_image_path = "cover.png"       # 1916×1076 (for splash, also used for rotating)
cover_image_path = "image.png"       # 1908×1072
run_image_path = "Doll_Run-removebg-preview.png"
watch_image_path = "Doll_Watch-removebg-preview.png"

# Thresholds
similarity_threshold = 0.95
distance_threshold = 350

###############################################################################
#                              UTILITY FUNCTIONS                              #
###############################################################################

def compute_similarity_fast(P1, P2):
    if P1 is None or P2 is None or len(P1) == 0 or len(P2) == 0:
        return 1.0, 0
    centroid_1 = np.mean(P1, axis=0)
    centroid_2 = np.mean(P2, axis=0)
    centroid_distance = np.linalg.norm(centroid_1 - centroid_2)

    std_1 = np.std(P1, axis=0)
    std_2 = np.std(P2, axis=0)
    norm_std1 = np.linalg.norm(std_1)
    norm_std2 = np.linalg.norm(std_2)
    if norm_std1 < 1e-8 or norm_std2 < 1e-8:
        similarity_distribution = 1.0
    else:
        similarity_distribution = np.dot(std_1, std_2) / (norm_std1 * norm_std2 + 1e-8)

    similarity_score = similarity_distribution / (1 + centroid_distance)
    movement_distance = (1 - similarity_score) * 100
    return similarity_score, movement_distance

def process_frame(pcap_file, two_players):
    cloud_arrays = []
    for _, points in vd.read_pcap(pcap_file):
        cloud_arrays.append(points)
    if not cloud_arrays:
        return None, None

    points = np.vstack(cloud_arrays)[:, :3]
    if two_players:
        mask_p1 = (
            (points[:, 0] <= 2.5) & (points[:, 0] >= 1.5) &
            (points[:, 2] >= -1) & (points[:, 2] <= 0.5) &
            (points[:, 1] >= -0.75) & (points[:, 1] <= -0.25)
        )
        mask_p2 = (
            (points[:, 0] <= 2.5) & (points[:, 0] >= 1.5) &
            (points[:, 2] >= -1) & (points[:, 2] <= 0.5) &
            (points[:, 1] >= 0.25) & (points[:, 1] <= 0.75)
        )
        return points[mask_p1], points[mask_p2]
    else:
        mask = (
            (points[:, 0] <= 2.5) & (points[:, 0] >= 1.5) &
            (points[:, 2] >= -1) & (points[:, 2] <= 0.5) &
            (points[:, 1] >= -0.5) & (points[:, 1] <= 0.5)
        )
        return points[mask], None

###############################################################################
#                           GAME THREAD LOGIC                                 #
###############################################################################

def game_thread_func(app):
    exit_flag.clear()
    pygame.mixer.music.stop()

    # Create LiDAR sockets
    sockets = [(socket.socket(socket.AF_INET, socket.SOCK_DGRAM), port) for port in UDP_PORTS]
    try:
        for sock, port in sockets:
            sock.bind((UDP_IP, port))
            sock.setblocking(0)
    except Exception as e:
        print(f"Socket binding failed: {e}")
        app.on_game_end(stopped=True)
        return

    two_players = (app.num_players.get() == 2)
    if two_players:
        os.makedirs(PLAYER1_FOLDER, exist_ok=True)
        os.makedirs(PLAYER2_FOLDER, exist_ok=True)
    else:
        os.makedirs(PLAYER1_FOLDER, exist_ok=True)

    P1_prev, P2_prev = None, None
    dist_p1, dist_p2 = 0, 0
    frame_counter = 1

    while not exit_flag.is_set():
        # ---------------- RUN!! -------------
        try:
            pygame.mixer.music.load(GAME_SOUND)
            pygame.mixer.music.play(-1)
        except Exception as e:
            print(f"Error playing game sound: {e}")

        app.update_status("Run!!")
        app.update_game_image(run_image_path)

        run_start = time.time()
        while time.time() - run_start < 8:
            if exit_flag.is_set():
                break
            time.sleep(0.1)

        pygame.mixer.music.stop()
        if exit_flag.is_set():
            break

        # --------------- WATCH!! -------------
        app.update_status("Watch!!")
        app.update_game_image(watch_image_path)
        watch_duration = random.randint(5, 10)
        watch_start = time.time()

        while (time.time() - watch_start < watch_duration) and not exit_flag.is_set():
            frames = {port: [] for port in UDP_PORTS}
            interval_start = time.time()

            while time.time() - interval_start < 0.12:
                for sock, port in sockets:
                    if exit_flag.is_set():
                        break
                    try:
                        data, _ = sock.recvfrom(1248)
                        frames[port].append(Ether(HEADERS[port] + data))
                    except BlockingIOError:
                        continue
                if exit_flag.is_set():
                    break

            # process frames
            for sock, port in sockets:
                if exit_flag.is_set():
                    break

                fpath_p1 = os.path.join(PLAYER1_FOLDER, f"frame_{frame_counter}.pcap")
                wrpcap(fpath_p1, frames[port])

                P1_new, P2_new = process_frame(fpath_p1, two_players)

                # Player 1
                sim_p1, d_p1 = compute_similarity_fast(P1_prev, P1_new)
                dist_p1 += d_p1
                print(f"Player 1 - Similarity={sim_p1:.3f}, Dist={dist_p1:.2f}")

                if not two_players:
                    app.update_info_label(sim_p1, dist_p1)

                if sim_p1 < similarity_threshold:
                    app.show_game_over_animation(1, win=False)
                    exit_flag.set()
                    break
                if dist_p1 >= distance_threshold:
                    app.show_game_over_animation(1, win=True)
                    exit_flag.set()
                    break

                # Player 2
                if two_players:
                    sim_p2, d_p2 = compute_similarity_fast(P2_prev, P2_new)
                    dist_p2 += d_p2
                    print(f"Player 2 - Similarity={sim_p2:.3f}, Dist={dist_p2:.2f}")
                    app.update_info_label(sim_p1, dist_p1, sim_p2, dist_p2)

                    if sim_p2 < similarity_threshold:
                        app.show_game_over_animation(2, win=False)
                        exit_flag.set()
                        break
                    if dist_p2 >= distance_threshold:
                        app.show_game_over_animation(2, win=True)
                        exit_flag.set()
                        break

                state = app.status_label['text']
                app.update_plot(P1_new, P2_new, mode=state.lower())

                P1_prev, P2_prev = P1_new, P2_new
                frame_counter += 1

    for sock, _ in sockets:
        sock.close()

    app.on_game_end(stopped=not exit_flag.is_set())

###############################################################################
#                           MAIN GAME APP                                     #
###############################################################################

class SquidLiDApp:
    def __init__(self, root):
        self.root = root
        self.root.title("SquidLiD - Real Game Interface")

        self.num_players = tk.IntVar(value=1)
        self.dist_p1 = 0
        self.dist_p2 = 0

        # Main frame
        self.main_frame = tk.Frame(root)
        self.main_frame.pack(fill=tk.BOTH, expand=True)

        # Left: info + progress
        self.left_frame = tk.Frame(self.main_frame)
        self.left_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=False, padx=10, pady=10)

        self.info_label = tk.Label(
            self.left_frame,
            text="Distances and Similarities:\n\n",
            justify=tk.LEFT,
            font=("Helvetica", 14)
        )
        self.info_label.pack(anchor=tk.NW)

        self.progress_p1 = ttk.Progressbar(self.left_frame, orient=tk.HORIZONTAL, length=200, mode='determinate')
        self.progress_p1.pack(pady=5)

        self.progress_p2 = ttk.Progressbar(self.left_frame, orient=tk.HORIZONTAL, length=200, mode='determinate')
        self.progress_p2.pack(pady=5)

        # Right frame: rotating vertical-axis image + combos + 3D plot
        self.right_frame = tk.Frame(self.main_frame)
        self.right_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=10, pady=10)

        # rotating small logo
        self.logo_frame = tk.Frame(self.right_frame)
        self.logo_frame.pack(pady=10, anchor=tk.N)

        # "Bigger" small image for rotation, let's pick (200,120)
        self.orig_logo = Image.open(start_image_path).resize((200, 120), Resampling.LANCZOS)
        self.logo_orig_w, self.logo_orig_h = self.orig_logo.size
        self.rotate_angle = 0
        self.rotated_logo_label = tk.Label(self.logo_frame)
        self.rotated_logo_label.pack()
        self.rotate_logo_vertical()

        # Combos
        selection_frame = tk.Frame(self.right_frame)
        selection_frame.pack(pady=5, anchor=tk.N)

        ttk.Label(selection_frame, text="Number of Players:").pack(side=tk.LEFT, padx=5)
        player_combo = ttk.Combobox(selection_frame, textvariable=self.num_players, values=[1, 2], width=5)
        player_combo.pack(side=tk.LEFT, padx=5)

        # Buttons
        btn_frame = tk.Frame(self.right_frame)
        btn_frame.pack(pady=10, anchor=tk.N)

        self.start_button = ttk.Button(btn_frame, text="Start Game", command=self.start_game)
        self.start_button.pack(side=tk.LEFT, padx=5)

        self.stop_button = ttk.Button(btn_frame, text="Stop Game", command=self.stop_game, state=tk.DISABLED)
        self.stop_button.pack(side=tk.LEFT, padx=5)

        # Status label
        self.status_label = tk.Label(self.right_frame, text="", font=("Helvetica", 18, "bold"))
        self.status_label.pack(pady=10, anchor=tk.N)

        # 3D plot
        self.fig = plt.Figure(figsize=(2,2))
        self.ax = self.fig.add_subplot(projection='3d')
        self.ax.set_title("LiDAR Visualization")
        self.ax.set_xlabel("X (m)")
        self.ax.set_ylabel("Y (m)")
        self.ax.set_zlabel("Z (m)")

        self.canvas = FigureCanvasTkAgg(self.fig, master=self.right_frame)
        self.canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True, anchor=tk.N)

        # Large doll image at bottom
        self.image_label = tk.Label(self.main_frame)
        self.image_label.pack(side=tk.BOTTOM, fill=tk.X, pady=10)

        self.game_thread = None
        self.game_over_label = None

    def rotate_logo_vertical(self):
        """
        Rotate the logo around the vertical axis.
        We'll do a "card-flip" effect by adjusting the width with abs(cos(angle)).
        If cos(angle) < 0, we also FLIP_LEFT_RIGHT so it looks like the backside.
        """
        angle_rad = math.radians(self.rotate_angle)
        cos_val = math.cos(angle_rad)

        scale_x = abs(cos_val)
        new_w = int(self.logo_orig_w * scale_x)
        if new_w < 1:
            new_w = 1
        new_h = self.logo_orig_h

        # resize
        spun_img = self.orig_logo.resize((new_w, new_h), Resampling.LANCZOS)

        # If cos_val < 0 => flip horizontally
        if cos_val < 0:
            spun_img = spun_img.transpose(Image.FLIP_LEFT_RIGHT)

        self.rotate_angle = (self.rotate_angle + 10) % 360

        tk_img = ImageTk.PhotoImage(spun_img)
        self.rotated_logo_label.config(image=tk_img)
        self.rotated_logo_label.image = tk_img

        # schedule next update
        self.root.after(200, self.rotate_logo_vertical)

    def play_idle_music(self):
        """Play 2.mp3 in a loop (idle music)."""
        try:
            pygame.mixer.init()
        except Exception:
            pass
        try:
            pygame.mixer.music.load(IDLE_MUSIC)
            pygame.mixer.music.play(-1)
        except Exception as e:
            print(f"Error playing idle music: {e}")

    def start_game(self):
        if self.game_thread and self.game_thread.is_alive():
            return
        exit_flag.clear()
        self.start_button.config(state=tk.DISABLED)
        self.stop_button.config(state=tk.NORMAL)

        self.dist_p1 = 0
        self.dist_p2 = 0
        self.progress_p1["value"] = 0
        self.progress_p2["value"] = 0

        self.info_label.config(text="Distances and Similarities:\n\n")
        self.status_label.config(text="")

        self.game_thread = threading.Thread(target=game_thread_func, args=(self,))
        self.game_thread.start()

    def stop_game(self):
        exit_flag.set()
        if self.game_thread and self.game_thread.is_alive():
            self.game_thread.join(timeout=2.0)
        self.start_button.config(state=tk.NORMAL)
        self.stop_button.config(state=tk.DISABLED)
        self.status_label.config(text="Game Stopped")

        pygame.mixer.music.stop()
        self.play_idle_music()

    def on_game_end(self, stopped=False):
        self.start_button.config(state=tk.NORMAL)
        self.stop_button.config(state=tk.DISABLED)

        if stopped:
            self.status_label.config(text="Game Stopped")
        else:
            self.status_label.config(text="Game Over or Stopped")

        pygame.mixer.music.stop()
        self.play_idle_music()

        # reset after 7s
        def delayed_reset():
            time.sleep(7)
            if not self.game_thread or not self.game_thread.is_alive():
                self.reset_app()
        threading.Thread(target=delayed_reset, daemon=True).start()

    def reset_app(self):
        self.start_button.config(state=tk.NORMAL)
        self.stop_button.config(state=tk.DISABLED)
        self.status_label.config(text="")
        self.info_label.config(text="Distances and Similarities:\n\n")
        self.progress_p1["value"] = 0
        self.progress_p2["value"] = 0

        if self.game_over_label:
            self.game_over_label.destroy()
            self.game_over_label = None

    def update_status(self, text):
        self.status_label.config(text=text)
        self.root.update_idletasks()

    def update_game_image(self, image_path):
        try:
            pil_img = Image.open(image_path)
            pil_img = pil_img.resize((700, 500), Resampling.LANCZOS)
            self.tk_img = ImageTk.PhotoImage(pil_img)
            self.image_label.config(image=self.tk_img)
        except Exception as e:
            print(f"Error loading image: {e}")
        self.root.update_idletasks()

    def update_plot(self, P1, P2, mode="watch"):
        self.ax.clear()
        self.ax.set_title("LiDAR Visualization")
        self.ax.set_xlabel("X (m)")
        self.ax.set_ylabel("Y (m)")
        self.ax.set_zlabel("Z (m)")

        if P1 is not None and len(P1) > 0:
            self.ax.scatter(P1[:, 0], P1[:, 1], P1[:, 2], s=2, label='Player 1')
        if self.num_players.get() == 2 and P2 is not None and len(P2) > 0:
            self.ax.scatter(P2[:, 0], P2[:, 1], P2[:, 2], s=2, label='Player 2')

        self.ax.legend()
        self.canvas.draw_idle()
        self.root.update_idletasks()

    def update_info_label(self, sim_p1, dist_p1, sim_p2=None, dist_p2=None):
        text_str = "Distances and Similarities:\n\n"
        text_str += f"Player 1:\n  Similarity: {sim_p1:.3f}\n  Distance: {dist_p1:.2f}\n"
        p1_percent = min(100, (dist_p1 / distance_threshold) * 100)
        self.progress_p1["value"] = p1_percent

        if self.num_players.get() == 2 and sim_p2 is not None and dist_p2 is not None:
            text_str += f"\nPlayer 2:\n  Similarity: {sim_p2:.3f}\n  Distance: {dist_p2:.2f}\n"
            p2_percent = min(100, (dist_p2 / distance_threshold) * 100)
            self.progress_p2["value"] = p2_percent
        else:
            self.progress_p2["value"] = 0

        self.info_label.config(text=text_str)
        self.root.update_idletasks()

    def show_game_over_animation(self, player_number, win=False):
        if not self.game_over_label:
            self.game_over_label = tk.Label(self.root, font=("Helvetica", 10, "bold"), fg="red")
        else:
            self.game_over_label.config(font=("Helvetica", 10, "bold"), fg="red")

        if win:
            self.game_over_label.config(text=f"PLAYER {player_number} WINS!")
        else:
            self.game_over_label.config(text="GAME OVER!")

        self.game_over_label.place(relx=0.5, rely=0.5, anchor=tk.CENTER)

        for size in range(10, 60, 2):
            if exit_flag.is_set():
                break
            self.game_over_label.config(font=("Helvetica", size, "bold"))
            self.root.update_idletasks()
            time.sleep(0.05)

        if win:
            self.game_over_label.config(text=f"PLAYER {player_number} WINS!")
        else:
            self.game_over_label.config(text=f"GAME OVER!\nPlayer {player_number} lost")
        self.root.update_idletasks()


###############################################################################
#                   FULL-SIZE SPLASH + COVER + THEN MAIN GAME                 #
###############################################################################

def show_full_splash(root):
    """
    Show image.png for 10s, 1916×1076, while playing INTRO_MUSIC
    then remove and proceed to cover screen.
    """
    w, h = 1916, 1076
    root.geometry(f"{w}x{h}")
    splash_frame = tk.Frame(root, width=w, height=h)
    splash_frame.pack()

    # Load image
    try:
        img = Image.open(start_image_path)
        tk_img = ImageTk.PhotoImage(img)
    except Exception as e:
        print(f"Error opening {start_image_path}: {e}")
        # fallback
        try:
            pygame.mixer.init()
            pygame.mixer.music.load(INTRO_MUSIC)
            pygame.mixer.music.play(-1)
        except:
            pass
        time.sleep(10)
        if splash_frame.winfo_exists():
            splash_frame.destroy()
        pygame.mixer.music.stop()
        return

    label = tk.Label(splash_frame, image=tk_img)
    label.image = tk_img
    label.pack()

    # Play music
    try:
        pygame.mixer.init()
        pygame.mixer.music.load(INTRO_MUSIC)
        pygame.mixer.music.play(-1)
    except Exception as e:
        print(f"Error: {e}")

    root.update()
    time.sleep(10)  # hold for 10s

    # Done
    if splash_frame.winfo_exists():
        splash_frame.destroy()
    pygame.mixer.music.stop()


def show_cover_screen(root):
    """
    Show 'cover.png' (1908×1072) with a "Start" button.
    Only after clicking Start do we proceed to the actual game UI.
    """
    w, h = 1908, 1072
    root.geometry(f"{w}x{h}")

    cover_frame = tk.Frame(root, width=w, height=h)
    cover_frame.pack()

    # load cover
    try:
        cover_img = Image.open(cover_image_path)
        tk_cover = ImageTk.PhotoImage(cover_img)
    except Exception as e:
        print(f"Error opening {cover_image_path}: {e}")
        tk_cover = None

    label = tk.Label(cover_frame)
    label.pack()
    if tk_cover:
        label.config(image=tk_cover)
        label.image = tk_cover

    # place a Start button
    def on_start():
        if cover_frame.winfo_exists():
            cover_frame.destroy()

    start_btn = tk.Button(cover_frame, text="Start", font=("Helvetica", 20), command=on_start)
    start_btn.place(relx=0.5, rely=0.5, anchor=tk.CENTER)

    root.update()

    while cover_frame.winfo_exists():
        root.update()
        time.sleep(0.1)


def main():
    root = tk.Tk()
    root.title("Splash + Cover + Real Game")

    # 1) Show image.png for 10s
    show_full_splash(root)

    # 2) Show cover screen with "Start" button
    show_cover_screen(root)

    # 3) Now proceed to the real game interface
    app = SquidLiDApp(root)
    app.play_idle_music()  # idle music

    def on_close():
        exit_flag.set()
        if app.game_thread and app.game_thread.is_alive():
            app.game_thread.join(timeout=2.0)
        root.destroy()

    root.protocol("WM_DELETE_WINDOW", on_close)
    root.mainloop()


if __name__ == "__main__":
    main()

Player 1 - Similarity=1.000, Dist=0.00
Player 1 - Similarity=0.999, Dist=0.07
Player 1 - Similarity=0.999, Dist=0.12
Player 1 - Similarity=0.999, Dist=0.22
Player 1 - Similarity=0.997, Dist=0.48
Player 1 - Similarity=0.992, Dist=1.25
Player 1 - Similarity=0.992, Dist=2.10
Player 1 - Similarity=0.998, Dist=2.27
Player 1 - Similarity=0.999, Dist=2.34
Player 1 - Similarity=0.997, Dist=2.63
Player 1 - Similarity=0.997, Dist=2.97
Player 1 - Similarity=0.987, Dist=4.28
Player 1 - Similarity=0.974, Dist=6.84
Player 1 - Similarity=0.996, Dist=7.22
Player 1 - Similarity=0.967, Dist=10.56
Player 1 - Similarity=0.993, Dist=11.24
Player 1 - Similarity=0.996, Dist=11.67
Player 1 - Similarity=0.943, Dist=17.38
Player 1 - Similarity=0.984, Dist=19.00
Player 1 - Similarity=0.901, Dist=28.94


Exception in thread Thread-3 (delayed_reset):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\bilal\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\bilal\AppData\Local\Temp\ipykernel_3840\942940179.py", line 418, in delayed_reset
  File "C:\Users\bilal\AppData\Local\Temp\ipykernel_3840\942940179.py", line 422, in reset_app
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\tkinter\__init__.py", line 1675